In [3]:
import csv
import os
import sys
from nltk import pos_tag, WordNetLemmatizer
from typing import List
import spacy
import pandas as pd
import stanza
from typing import List

In [25]:
def extract_tokenized_sentences(file_path: str) -> List[List[str]]:
    """Extract a list of tokens for every sentence from the corpus and return it."""
    tokenized_sentences = []
    sentence_tokens = []

    with open(file_path, 'r', encoding='utf8') as infile:
        filereader = csv.reader(infile, delimiter='\t', quotechar='\\')
        for row in filereader:
            if len(row)==1:
                continue
            if not row:  # empty line, end of sentence
              
                tokenized_sentences.append(sentence_tokens)
                
                sentence_tokens = []
            else:
                token = row[1]
               
                sentence_tokens.append(token)

    tokenized_sentences.append(sentence_tokens)  # append the last sentence; there is no empty line at the end of file

    return tokenized_sentences

file = "data/en_ewt-up-train.conllu"
data= extract_tokenized_sentences(file)
for d in data:
    print(d)
    break


['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.']


In [17]:
file_path = "data/en_ewt-up-train.conllu"
with open(file_path, 'r', encoding='utf8') as infile:
    filereader = csv.reader(infile, delimiter='\t', quotechar='\\')
    for row in filereader:
        if len(row)==1:
            continue
        if not row:  # empty line, end of sentence

            # tokenized_sentences.append(sentence_tokens)

            sentence_tokens = []
        else:
            predicate = row[:11]
            print(predicate)
            break

['1', 'Al', 'Al', 'PROPN', 'NNP', 'Number=Sing', '0', 'root', '0:root', 'SpaceAfter=No', '_']


In [21]:
def generate_preprocessed_file(infile_path: str, outfile_path: str) -> None:
    """Generate a new file containing information obtained through preprocessing."""
    with open(outfile_path, 'w',  newline='', encoding='utf8') as outfile:
        filewriter = csv.writer(outfile, delimiter='\t', quotechar='\\')

        with open(infile_path, 'r', encoding='utf8') as infile:
            filereader = csv.reader(infile, delimiter='\t', quotechar='\\')
            

            sentence_index = 0
            token_index = 0
            for inrow in filereader:
                if len(inrow)==1:
                    continue
                if not inrow:  # empty line
                    filewriter.writerow(inrow)  # write an empty row to output file
                    
                else:
                    if token_index ==0:
                        filewriter.writerow(['id','token','lemma','pos_category','pos_tag', 
                                             'passive/active','head','dep','rel','after','predicate'])
                        predicate = inrow[10]
                        if predicate != "_":
                            predicate == "predicate"
                            
                        filewriter.writerow(inrow[:10]+[predicate])
                        token_index += 1
                    else:
                        if predicate != "_":
                            predicate == "predicate"
                        filewriter.writerow(inrow[:10]+[predicate])
                        


paths = ["data/en_ewt-up-train.conllu", "data/en_ewt-up-dev.conllu", "data/en_ewt-up-test.conllu"]

for path in paths:
    print(f'Preprocessing {os.path.basename(path)}')

    preprocessed_path = path.replace('.conllu', 'preprocessed.conllu')
    generate_preprocessed_file(path, preprocessed_path)



Preprocessing en_ewt-up-train.conllu
Preprocessing en_ewt-up-dev.conllu
Preprocessing en_ewt-up-test.conllu


In [29]:
input_file = "data/en_ewt-up-trainpreprocessed.conllu"
input_data = pd.read_csv(input_file, encoding='utf-8', sep='\t', keep_default_na=False,     
                             quotechar='\\', skip_blank_lines=False)
input_data

,id,token,lemma,pos_category,pos_tag,passive/active,head,dep,rel,after,predicate
0,1,Al,Al,PROPN,NNP,Number=Sing,0,root,0:root,SpaceAfter=No,_
1,2,-,-,PUNCT,HYPH,_,1,punct,1:punct,SpaceAfter=No,_
2,3,Zaman,Zaman,PROPN,NNP,Number=Sing,1,flat,1:flat,_,_
3,4,:,:,PUNCT,:,_,1,punct,1:punct,_,_
4,5,American,american,ADJ,JJ,Degree=Pos,6,amod,6:amod,_,_
...,...,...,...,...,...,...,...,...,...,...,...
217147,23,my,my,PRON,PRP$,Number=Sing|Person=1|Poss=Yes|PronType=Prs,24,nmod:poss,24:nmod:poss,_,_
217148,24,car,car,NOUN,NN,Number=Sing,21,obl,21:obl:on,SpaceAfter=No,_
217149,25,),),PUNCT,-RRB-,_,4,punct,4:punct,SpaceAfter=No,_
217150,26,.,.,PUNCT,.,_,4,punct,4:punct,_,_


In [ ]:
# run in terminal, if you don't have the package: python3 -m spacy download en_core_web_sm

# def read_parse_result(parse_path: str):
#     return pd.read_csv(parse_path, sep="\t", quotechar="|", encoding="utf-8")


# def prepare_data(content):
#     # with open(text_path, encoding="utf-8") as f:
#     #     content = f.read()

#     nlp = spacy.load("en_core_web_sm")
#     doc = nlp(content)

#     # https://spacy.io/api/token
#     token = [tok.text for tok in doc]
#     dependency = [tok.dep_ for tok in doc]
#     head = [tok.head for tok in doc]
#     dependent = [[t.text for t in tok.children] for tok in doc]
#     constituent = [[t.text for t in tok.subtree] for tok in doc]

#     parse_info = {"token": token, "dependency": dependency,
#                   "head": head, "dependent": dependent,
#                   "constituent": constituent}

#     df = pd.DataFrame.from_dict(parse_info)
    
#     return df

# def extract_head(dataframe) -> List[str]:
#     return dataframe["head"].to_list()


In [37]:
# data

In [ ]:
# meta_df= []
# for i in range(len(data)):
#     cols = [
#         "token", "dependency", "head", 'dependent', "consituent"
#     ]
#     datas = ' '.join(data[i])
#     datas = prepare_data(datas)
#     meta = pd.DataFrame(datas)
#     meta.columns = cols[0:]
#     meta_df.append(meta)
# print(meta_df)
    

In [ ]:
# inspired by https://github.com/cltl/ma-ml4nlp-labs/blob/main/code/assignment3/CRF.py, extract_sents_from_conll
